In [1]:
from misc_fun import FLAGS
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Input
from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard
from time import time
import os.path
from tensorflow.python.client import timeline
import os

In [2]:
""" 
    REMINDER: Each time you change the mode,
    you need to restart the Kernel by using the refresh botton
    (may) next to the stop botton.

    Choose a Model to Run
    You Can Either Choose from:
    1: A CNN with multiple convolution layers and two dense layers. (Work as a Control Group)
    2: A CNN contains dropout layers
    3: A CNN with smaller kernel and smaller layers
    4: A CNN with bias in some of the layers
    5: A 2-layer fully connected neural network without any convolution layers
"""

MODE = 5


In [3]:
# import and pre-processing data
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images /= 255
test_images /= 255
train_labels = keras.utils.to_categorical(train_labels, 10)
test_labels = keras.utils.to_categorical(test_labels, 10)

In [4]:
# construct model with multiple layers （As Control Group)
if 1 == MODE:
    model_CNN_multiple_layer = Sequential()
    # convolution layer 1: 5*5 kernel is divided into 2* (3*3) kennels
    model_CNN_multiple_layer.add(Conv2D(
        filters=32,
        kernel_size=(3, 3),
        padding="same",
        activation='relu',
        input_shape=input_shape
    ))
    model_CNN_multiple_layer.add(Conv2D(
        filters=32,
        kernel_size=(3, 3),
        padding="same",
        activation='relu'))
    # pooling layer 1: max pooling method
    model_CNN_multiple_layer.add(MaxPooling2D(pool_size=(2, 2)))
    # convolution layer 2: 5*5 kernel is divided into 2* (3*3) kennels
    model_CNN_multiple_layer.add(Conv2D(
        filters=64,
        kernel_size=(3, 3),
        padding="same",
        activation='relu',
    ))
    model_CNN_multiple_layer.add(Conv2D(
        filters=64,
        kernel_size=(3, 3),
        padding="same",
        activation='relu',
    ))
    # pooling layer 1: max pooling method
    model_CNN_multiple_layer.add(MaxPooling2D(pool_size=(2, 2)))
    
    model_CNN_multiple_layer.add(Flatten())
    model_CNN_multiple_layer.add(Dense(1024, activation='relu'))
    model_CNN_multiple_layer.add(Dense(10, activation='softmax')) 


In [5]:
if 2 == MODE:
    # construct a CNN model with dropout layers
    
    model_CNN_with_dropout = Sequential()
    # convolution layer 1: 5*5 kernel is divided into 2* (3*3) kennels
    model_CNN_with_dropout.add(Conv2D(
        filters=32,
        kernel_size=(3, 3),
        padding="same",
        activation='relu',
        input_shape=input_shape
    ))
    model_CNN_with_dropout.add(Conv2D(
        filters=32,
        kernel_size=(3, 3),
        padding="same",
        activation='relu'))
    # pooling layer 1: max pooling method
    model_CNN_with_dropout.add(MaxPooling2D(pool_size=(2, 2)))
    # add a drop out layer
    model_CNN_with_dropout.add(Dropout(0.2))
    # convolution layer 2: 5*5 kernel is divided into 2* (3*3) kennels
    model_CNN_with_dropout.add(Conv2D(
        filters=64,
        kernel_size=(3, 3),
        padding="same",
        activation='relu',
    ))
    model_CNN_with_dropout.add(Conv2D(
        filters=64,
        kernel_size=(3, 3),
        padding="same",
        activation='relu',
    ))
    # pooling layer 1: max pooling method
    model_CNN_with_dropout.add(MaxPooling2D(pool_size=(2, 2)))
    # add a drop out layer
    model_CNN_with_dropout.add(Dropout(0.2))
    # 
    model_CNN_with_dropout.add(Flatten())
    model_CNN_with_dropout.add(Dense(1024, activation='relu'))
    model_CNN_with_dropout.add(Dropout(0.2))
    model_CNN_with_dropout.add(Dense(10, activation='softmax'))


In [6]:
if 3 == MODE:
    # construct a CNN model with smaller kernel and fewer layers
    
    model_CNN_smaller_kernel = Sequential()
    # convolution layer 1: 3*3 kernel 
    model_CNN_smaller_kernel.add(Conv2D(
        filters=32,
        kernel_size=(3, 3),
        padding="same",
        activation='relu',
        input_shape=input_shape
    ))
    # pooling layer 1: max pooling method
    model_CNN_smaller_kernel.add(MaxPooling2D(pool_size=(2, 2)))
    # convolution layer 2: 3*3 kernel
    model_CNN_smaller_kernel.add(Conv2D(
        filters=64,
        kernel_size=(3, 3),
        padding="same",
        activation='relu',
    ))
    model_CNN_smaller_kernel.add(Flatten())
    model_CNN_smaller_kernel.add(Dense(1024, activation='relu'))
    model_CNN_smaller_kernel.add(Dense(10, activation='softmax'))


In [7]:
if 4 == MODE:
    #contruct a model with bias inside each of the layer
    
    model_CNN_with_bias = Sequential()
    # convolution layer 1: 5*5 kernel is divided into 2* (3*3) kennels
    model_CNN_with_bias.add(Conv2D(
        filters=32,
        kernel_size=(3, 3),
        padding="same",
        activation='relu',
        input_shape=input_shape,
        use_bias=True
    ))
    model_CNN_with_bias.add(Conv2D(
        filters=32,
        kernel_size=(3, 3),
        padding="same",
        activation='relu',
        use_bias=True
    ))
    # pooling layer 1: max pooling method
    model_CNN_with_bias.add(MaxPooling2D(pool_size=(2, 2)))
    # convolution layer 2: 5*5 kernel is divided into 2* (3*3) kennels
    model_CNN_with_bias.add(Conv2D(
        filters=64,
        kernel_size=(3, 3),
        padding="same",
        activation='relu',
        use_bias=True
    ))
    model_CNN_with_bias.add(Conv2D(
        filters=64,
        kernel_size=(3, 3),
        padding="same",
        activation='relu',
        use_bias=True
    ))
    # pooling layer 1: max pooling method
    model_CNN_with_bias.add(MaxPooling2D(pool_size=(2, 2)))
    
    model_CNN_with_bias.add(Flatten())
    model_CNN_with_bias.add(Dense(1024, activation='relu',use_bias=True))
    model_CNN_with_bias.add(Dense(10, activation='softmax'))


In [8]:
if 5 == MODE:
    # construct a model without convolution layers
    model_fully_connected = Sequential()
    model_fully_connected.add(Flatten())
    model_fully_connected.add(Dense(1024, activation='relu'))
    model_fully_connected.add(Dense(10, activation='softmax'))


In [9]:
# define storage path for models
multiple_layer_path = os.path.join(FLAGS.DEFAULT_OUT,'model_multiple_layers/')
dropout_path = os.path.join(FLAGS.DEFAULT_OUT,'model_with_dropout/')
smaller_kernel_path = os.path.join(FLAGS.DEFAULT_OUT,'model_smaller_kernel/')
bias_path = os.path.join(FLAGS.DEFAULT_OUT,'model_with_bias/')
fully_connected_path = os.path.join(FLAGS.DEFAULT_OUT,'model_fully_connected/')


In [10]:
# construct tensorboard event files for models
# define the paths for event files
multi_layer_tensorboard = os.path.join(multiple_layer_path, 'logdir/{}'.format(time()))
dropout_tensorboard = os.path.join(dropout_path, 'logdir/{}'.format(time()))
smaller_kernel_tensorboard = os.path.join(smaller_kernel_path, 'logdir/{}'.format(time()))
bias_tensorboard = os.path.join(bias_path, 'logdir/{}'.format(time()))
fully_connected_tensorboard = os.path.join(fully_connected_path, 'logdir/{}'.format(time()))

#create tensorboards 
# (write_images will write all images into the Tensorboard files,
# which will significantly increase the size of the files,
# and will not be used in this project unless required)
if 1 == MODE:
    tensorboard_multi = TensorBoard(log_dir=multi_layer_tensorboard.format(time()),
                                    write_graph=True,
                                    write_grads=True,
                                    histogram_freq=1,
                                    # write_images=True 
                                    )
elif 2 == MODE:
    tensorboard_dropout = TensorBoard(log_dir=dropout_tensorboard.format(time()),
                                      write_graph=True,
                                      write_grads=True,
                                      histogram_freq=1)
elif 3 == MODE:
    tensorboard_smaller = TensorBoard(log_dir=smaller_kernel_tensorboard.format(time()),
                                      write_graph=True,
                                      write_grads=True,
                                      histogram_freq=1)
elif 4 == MODE:
    tensorboard_bias = TensorBoard(log_dir=bias_tensorboard.format(time()),
                                   write_graph=True,
                                   write_grads=True,
                                   histogram_freq=1)
elif 5 == MODE:
    tensorboard_fully = TensorBoard(log_dir=fully_connected_tensorboard.format(time()),
                                    write_graph=True,
                                    write_grads=True,
                                    histogram_freq=1)
else:
    print('Wrong MODE, Check it!')
    os._exit()


In [11]:
# collect information for Chrome timeline json files
run_options = tf.RunOptions(trace_level = tf.RunOptions.FULL_TRACE)
run_metadata = tf.RunMetadata()

In [12]:
# Compile model
if 1 == MODE:
    model_CNN_multiple_layer.compile(loss=keras.losses.categorical_crossentropy,
                                     optimizer=keras.optimizers.Adadelta(),
                                     metrics=['accuracy'],
                                     # options=run_options,
                                     run_metadata=run_metadata)
elif 2 == MODE:
    model_CNN_with_dropout.compile(loss=keras.losses.categorical_crossentropy,
                                   optimizer=keras.optimizers.Adadelta(),
                                   metrics=['accuracy'],
                                   run_metadata=run_metadata)
elif 3 == MODE:
    model_CNN_smaller_kernel.compile(loss=keras.losses.categorical_crossentropy,
                                     optimizer=keras.optimizers.Adadelta(),
                                     metrics=['accuracy'],
                                     # options=run_options,
                                     run_metadata=run_metadata)
elif 4 == MODE:
    model_CNN_with_bias.compile(loss=keras.losses.categorical_crossentropy,
                                optimizer=keras.optimizers.Adadelta(),
                                metrics=['accuracy'],
                                run_metadata=run_metadata)
elif 5 == MODE:
    model_fully_connected.compile(loss=keras.losses.categorical_crossentropy,
                                  optimizer=keras.optimizers.Adadelta(),
                                  metrics=['accuracy'],
                                  run_metadata=run_metadata)


Instructions for updating:
Colocations handled automatically by placer.


In [13]:
# add check points
if 1 == MODE:
    multiple_layer_checkpoint = ModelCheckpoint(
        filepath=os.path.join(multiple_layer_path,'checkpointfiles/cp-{epoch:04d}.ckpt'),
        verbose=1,
        save_weights_only=True,
        save_best_only=False,
        period=2)
elif 2 == MODE:
    dropout_checkpoint = ModelCheckpoint(
        filepath=os.path.join(dropout_path,'checkpointfiles/cp-{epoch:04d}.ckpt'),
        verbose=1,
        save_weights_only=True,
        save_best_only=False,
        period=2)
elif 3 == MODE:
    smaller_kernel_checkpoint = ModelCheckpoint(
        filepath=os.path.join(smaller_kernel_path,'checkpointfiles/cp-{epoch:04d}.ckpt'),
        verbose=1,
        save_weights_only=True,
        save_best_only=False,
        period=2)
elif 4 == MODE:
    bias_checkpoint = ModelCheckpoint(
        filepath=os.path.join(bias_path,'checkpointfiles/cp-{epoch:04d}.ckpt'),
        verbose=1,
        save_weights_only=True,
        save_best_only=False,
        period=2)
elif 5 == MODE:
    fully_connected_checkpoint = ModelCheckpoint(
        filepath=os.path.join(fully_connected_path,'checkpointfiles/cp-{epoch:04d}.ckpt'),
        verbose=1,
        save_weights_only=True,
        save_best_only=False,
        period=2)


In [14]:
# fit the model
if 1 == MODE:
    model_CNN_multiple_layer.fit(train_images, train_labels,
                                 batch_size=128,
                                 epochs=12,
                                 verbose=0,
                                 validation_data=(test_images, test_labels),
                                 callbacks=[tensorboard_multi, multiple_layer_checkpoint],
                                 )
elif 2 == MODE:
    model_CNN_with_dropout.fit(train_images, train_labels,
                               batch_size=128,
                               epochs=12,
                               verbose=0,
                               validation_data=(test_images, test_labels),
                               callbacks=[tensorboard_dropout, dropout_checkpoint],
                               )
elif 3 == MODE:
    model_CNN_smaller_kernel.fit(train_images, train_labels,
                                 batch_size=128,
                                 epochs=12,
                                 verbose=0,
                                 validation_data=(test_images, test_labels),
                                 callbacks=[tensorboard_smaller, smaller_kernel_checkpoint],
                                 )
elif 4 == MODE:
    model_CNN_with_bias.fit(train_images, train_labels,
                            batch_size=128,
                            epochs=12,
                            verbose=0,
                            validation_data=(test_images, test_labels),
                            callbacks=[tensorboard_bias, bias_checkpoint],
                            )
elif 5 == MODE:
    model_fully_connected.fit(train_images, train_labels,
                              batch_size=128,
                              epochs=12,
                              verbose=0,
                              validation_data=(test_images, test_labels),
                              callbacks=[tensorboard_fully, fully_connected_checkpoint]
                              )
print('FIT FINISHED')


Instructions for updating:
Use tf.cast instead.



Epoch 00002: saving model to E:/Artificial_Intelligence_for_Mechatronics/Assignment1/results/model_fully_connected/checkpointfiles/cp-0002.ckpt

Consider using a TensorFlow optimizer from `tf.train`.


Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.



Epoch 00004: saving model to E:/Artificial_Intelligence_for_Mechatronics/Assignment1/results/model_fully_connected/checkpointfiles/cp-0004.ckpt

Consider using a TensorFlow optimizer from `tf.train`.



Epoch 00006: saving model to E:/Artificial_Intelligence_for_Mechatronics/Assignment1/results/model_fully_connected/checkpointfiles/cp-0006.ckpt

Consider using a TensorFlow optimizer from `tf.train`.



Epoch 00008: saving model to E:/Artificial_Intelligence_for_Mechatronics/Assignment1/results/model_fully_connected/checkpointfiles/cp-0008.ckpt

Consider using a TensorFlow optimizer from `tf.train`.



Epoch 00010: saving model to E:/Artificial_Intelligence_for_Mechatronics/Assignment1/results/model_fully_connected/checkpointfiles/cp-0010.ckpt

Consider using a TensorFlow optimizer from `tf.train`.



Epoch 00012: saving model to E:/Artificial_Intelligence_for_Mechatronics/Assignment1/results/model_fully_connected/checkpointfiles/cp-0012.ckpt

Consider using a TensorFlow optimizer from `tf.train`.


FIT FINISHED


In [15]:
if 1 == MODE:
    score = model_CNN_multiple_layer.evaluate(test_images, test_labels, verbose=0)
elif 2 == MODE:
    score = model_CNN_with_dropout.evaluate(test_images, test_labels, verbose=0)
elif 3 == MODE:
    score = model_CNN_smaller_kernel.evaluate(test_images, test_labels, verbose=0)
elif 4 == MODE:
    score = model_CNN_with_bias.evaluate(test_images, test_labels, verbose=0)
elif 5 == MODE:
    score = model_fully_connected.evaluate(test_images, test_labels, verbose=0)

method_name = ['Multiple Layer CNN Model',
               'CNN Model with Dropout',
               'CNN Model with Smaller Kernel and Less Layers',
               'CNN Model with Bias',
               'Fully Connected Neural Network']
print('Test Loss for {} is:       '.format(method_name[MODE-1]), score[0])
print('Test Accuracy for {} is: '.format(method_name[MODE-1]), score[1])

Test Loss for Fully Connected Neural Network is:        0.31421863079071044
Test Accuracy for Fully Connected Neural Network is:  0.8906


In [16]:
# write the .json file
# There is an unsolved issue that the program requires a cupti64_90.dll
# when running on a NVIDIA GPU. However,this dll has been removed from the CUDA library
# after CUDA 8.0 version.
trace = timeline.Timeline(step_stats=run_metadata.step_stats)
if 1 == MODE:
    json_path = os.path.join(multiple_layer_path, 'model.json')
elif 2 == MODE:
    json_path = os.path.join(dropout_path, 'model.json')
elif 3 == MODE:
    json_path = os.path.join(smaller_kernel_path, 'model.json')
elif 4 == MODE:
    json_path = os.path.join(bias_path, 'model.json')
else:
    json_path = os.path.join(fully_connected_path, 'model.json')
with open(json_path, 'w')as json_file:
    json_file.write(trace.generate_chrome_trace_format())
